## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-14-10-52-39 +0000,bbc,Israeli minister announces settlement plans 't...,https://www.bbc.com/news/articles/ckgdzxpkdd7o...
1,2025-08-14-10-51-47 +0000,nyt,What to Know About the Heat Wave in Southern E...,https://www.nytimes.com/2025/08/12/world/europ...
2,2025-08-14-10-46-20 +0000,nypost,Tattooed maniac allegedly beat his father to d...,https://nypost.com/2025/08/14/us-news/north-ca...
3,2025-08-14-10-46-15 +0000,bbc,EU to send wildfire help to Spain as death tol...,https://www.bbc.com/news/articles/ckg4k7ppmnko...
4,2025-08-14-10-27-29 +0000,nypost,Virginia Beach passes resolution to ban ‘vulga...,https://nypost.com/2025/08/14/us-news/virginia...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2315/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
45,trump,58
110,putin,19
250,new,17
85,russia,12
49,ukraine,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
230,2025-08-13-17-57-20 +0000,nypost,Trump says he doesn’t believe he can convince ...,https://nypost.com/2025/08/13/us-news/trump-sa...,119
210,2025-08-13-19-04-10 +0000,nyt,Trump’s Affinity for Putin Will Be Tested at H...,https://www.nytimes.com/2025/08/13/us/politics...,116
273,2025-08-13-15-03-41 +0000,nypost,Trump vows main goal of Putin Alaska summit is...,https://nypost.com/2025/08/13/world-news/trump...,113
58,2025-08-14-08-52-43 +0000,nypost,Zelensky meets with UK’s Starmer as Europe bra...,https://nypost.com/2025/08/14/world-news/ukrai...,113
190,2025-08-13-20-35-00 +0000,wapo,"Europeans, Zelensky cite unity with Trump ahea...",https://www.washingtonpost.com/world/2025/08/1...,112


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
230,119,2025-08-13-17-57-20 +0000,nypost,Trump says he doesn’t believe he can convince ...,https://nypost.com/2025/08/13/us-news/trump-sa...
11,54,2025-08-14-10-01-36 +0000,bbc,Zelensky meets Starmer ahead of US-Russia summ...,https://www.bbc.com/news/articles/cp37vg34g8jo...
94,54,2025-08-14-04-11-33 +0000,nypost,Ex-police chief warns Chicago cops will get hu...,https://nypost.com/2025/08/14/us-news/ex-polic...
255,52,2025-08-13-16-25-00 +0000,wsj,New York Attorney General Letitia James is sui...,https://www.wsj.com/politics/policy/new-york-a...
260,40,2025-08-13-16-03-42 +0000,nypost,"Trump announces George Strait, Sylvester Stall...",https://nypost.com/2025/08/13/us-news/trump-an...
102,38,2025-08-14-03-00-00 +0000,wsj,As other Asian countries ramp up manufacturing...,https://www.wsj.com/economy/trade/trump-tariff...
70,37,2025-08-14-06-37-28 +0000,nypost,US fugitive Nicholas Rossi accused of faking h...,https://nypost.com/2025/08/14/us-news/fugitive...
211,34,2025-08-13-19-04-00 +0000,wsj,President Trump’s apocalyptic portrait of Wash...,https://www.wsj.com/politics/policy/trumps-d-c...
100,34,2025-08-14-03-16-56 +0000,nypost,"Woman who allegedly murdered NYC girlfriend, c...",https://nypost.com/2025/08/13/us-news/girlfrie...
297,32,2025-08-13-13-07-00 +0000,wsj,Air Canada to Suspend Flights as Flight Attend...,https://www.wsj.com/business/airlines/air-cana...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
